# **Carga de cartografías en Python**
En este cuaderno vamos a ver como cargar cartografías y hacer representaciones simples de las variables sobre mapas. Antes de comenzar es conveninte, aunque no estrictamente necesario, crear un nuevo entorno para instalar las librerías espaciales con sus dependencias

In [1]:
# Podemos Crear un nuevo entorno de trabajo (mejor desde el terminal)
#! conda create -n geo_env
#! conda activate geo_env
#! conda config --env --add channels conda-forge
#! conda config --env --set channel_priority strict

# conda install --channel conda-forge geopandas

**Bibliografía recomendada**: La referencia básica para trabajar con datos espaciales con python es [Geographic Data Science with Python](https://geographicdata.science/book/intro.html)

Las principales librerías para trabajar con cartografías y datos espaciales son:
- `geopandas` para trabajar con datos espaciales. Trabaja con GeoDataFrame o geotablas, que es un dataframe donde cada fila corresponde a una figura geométrica, cada columna es un metadato y con una columna final con la geometría, puntos (x,y) que hay que unir para definir cada figura espacial (sigue la estructura de "simple features" utilizada también en `PostGis` de `Postgres` o `sf` de R)   
- `xarray` para datos espaciales raster, que representan superficies mediante cuadrículas. Trabaja con objetos DataArrays donde cada elemento tiene una coordenada (x,y) que correpnde a una celda. Yo en esta clase no trabajaré con estos datos raster. Para más información de como trabajar con datos raster, y lo más importante, de como poder convertirlos en datos tabulares tipo GeoDataFrame para poder aplicar todas las librerías de sci-kit.learn (por ejemplo) para hacer machine learning véase [Geographic Data Science with Python](https://geographicdata.science/book/intro.html)

<br>

- Para hacer el análisis estadístico espacial utilizaremos la librería **`PySal`** (sobre esta metalibrería está construido, por ejemplo [Qgis](https://www.qgis.org/)). Sobre esta librería hablaremos un poco más adelante, pero si alguien está interesado puede consultar [pysal.org](https://pysal.org/)
- para gestionar cartografías, fusionar unir, crear zonas, etc puede utilizarse  la librería **`Shapely`**. Yo no voy a ver nada de esta librería, pero animo a toda persona interesada a consultar el manual de referencia de esta librería [Shapely](https://shapely.readthedocs.io/en/stable/index.html)


## **Trabajando con Geopandas** 

La librería `geopandas` permite trabajar con cartografías (con mapas) utilizando la arquitectura de los data frames de `pandas`. Para una introducción a esta librería `geopandas` es muy recomendable visitar su web [GeoPandas](https://geopandas.org/en/stable/getting_started/introduction.html). 

Importamos Geopandas y otras librerías que también utilizaremos

In [ ]:
import geopandas as gpd

import pandas as pd
import matplotlib.pyplot as plt


`GeoPandas` extiende la librería `Pandas` para que pueda trabajar con datos espaciales. La estructura de datos principal en `GeoPandas` es `geopandas.GeoDataFrame`, una subclase de `pandas.DataFrame`, que puede almacenar **columnas con la geometría** de cada objeto espacial (almacenado como una fila del dataframe) y realizar operaciones espaciales.

Esa columna con las geometrías es una `geopandas.GeoSeries`, una subclase de `pandas.Series`, que almacena las geometrías, esto es los puntos en el espacio que conforman de cada objeto espacial (ya sea un punto, una linea o un polígono espacial).   

Por lo tanto, un `GeoDataFrame` es una combinación de `pandas.Series`, con datos tradicionales (numéricos, booleanos, texto, etc.), y `geopandas.GeoSeries`, con geometrías (puntos, polígonos, etc.).    

Cada `GeoSeries` contiene una geometría y tiene el atributo `GeoSeries.crs`, la cual almacena información sobre la proyección utilizada en la geometría (CRS son las siglas en inglés de **Coordinate Reference System**)

## Lectura de Ficheros de Cartografías

Los mapas, las cartografías, hay que cargarlas, bien utilizando algún paquete que incorpore como dato algún mapa (sobre todo mapas mundiales), bien cargando nuestras propias cartografíac.

Cuando se trabajan con datos espaciales se suele trabajar directamente con cartografías, es decir, suelen leerse y cargarse los mapas desde **ficheros con las cartografías**. Estos ficheros con las cortagrafías (mapas), incluyen metadatos de cada objeto espacial y su geometría. El formato de estos ficheros de cartografías puede ser variado (por ejemplo, GeoPackage, GeoJSON, Shapefile). Yo os he dejado una serie de cartografías en formato `shape`.

Pueden descargarse cartografías

- desde organismos oficiales como [Eurostat] (https://ec.europa.eu/eurostat/web/gisco/geodata/statistical-units/territorial-units-statistics)
- o como los insitutos nacionales de Estadística y Geografía como el de México [INEGI] (https://www.inegi.org.mx/temas/mg/#descargas) 
- o desde páginas espcecializada en recopilar cartografías públicas como [Natural Earth] (https://www.naturalearthdata.com/downloads/)

La lectura de los ficheros de cartografías se realiza utilizando `geopandas.read_file()`, que detecta automáticamente el tipo de archivo y crea un `GeoDataFrame`

In [ ]:
gdf =gpd.read_file("cartografias/CCAA_GEO_ETRS89.shp")
gdf.info()



In [ ]:
gdf

Podemos hacer la representación de nuestro primer mapa geopolítica simplement con el método plot asociado al geodataframe

In [ ]:
gdf.plot()

Las geometrías puede ser de tipo Punto, Líneas o polígonos espaciales (simple features). Cada figura espacial (**feature**, !!!ojo que en machine learning feature sería cada variable o característica,columna, correspondiente a cada unidad, a cada fila, pero aquí feature es un objeto espacial, una unidad de observación o una fila!!!! ).    
    
    ... como digo cada objeto, unidad o figura espacial puede estar compuesta por ejemplo por un único polígono, como el caso de Asturias

In [ ]:
gdf["geometry"][2]


o como conjunto de polígonos o Multipolígonos, como el caso de las Islas Canarias

In [ ]:
gdf["geometry"][4]

En este caso la cartografía venía vacía, sin metadatos, podemos ir añadiendo sólo algunas variables que podemos obtener de la propia geometría, como el área total de cada CCAA,o su centroide

In [ ]:
gdf["area"] = gdf.area

In [ ]:
gdf.head()

Para el calculo del área es necesario tener bien definido el sistema de referencia, la proyección que se está utilizando para la localización de los diferentes elementos del mapa que estamos representando (los polígonos de las Comunidades Autónomas)
Es posible que os aparezca un warning referente a estas proyecciones utilizadas en las geometrías. En este caso yo he bajado los datos cartográficos del INE (EPSG:4258)

Para consultar características de cada sistema de referencia
[http://epsg.io]
[http://spatialreference.org]
CRS("epsg:3857") #PSeudomercator Este es el que usa Open Street Maps
http://spatialreference.org/ref/epsg/3857/
Projection used in many popular web mapping applications (Google/Bing/OpenStreetMap/etc). Sometimes known as EPSG:900913.
EPSG:4326 WGS 84

  

In [ ]:
gdf.crs


In [10]:
# Podría fijar la cartografía en otro sistema de referencia:
# gdf=gdf.to_crs(epsg=3857) # este epsg=3857 Pseudo-Mercator -- Spherical Mercator es el que utilizan Google Maps, OpenStreetMap, Bing, ArcGIS, ESRI

Calculo ahora el centroide de cada CCAA y lo guardo como otra columna adicional en mi datagrame

In [ ]:
gdf['centroide'] = gdf.centroid

In [ ]:
gdf.head()

O podemos calcular la distancia a un punto (por ejemplo al centroide de Madrid CA13)

In [ ]:
gdf["geometry"][12]

In [ ]:
Madrid_point = gdf['centroide'].iloc[12]
gdf['distancia'] = gdf['centroide'].distance(Madrid_point)


In [ ]:
gdf

### Podemos hacer gráficos de cloropletas estáticos (aquí los colores están en función del área)

In [ ]:
gdf.plot("area", legend=True)

Es muy importante la paleta de colores, que las tonalidades muestren claramente una relación entre intensidad y valores de las variables [Paletas de Colores en matplotlib](https://matplotlib.org/stable/tutorials/colors/colormaps.html)


In [ ]:
gdf.plot("area", legend=True, cmap='Greys', figsize=(10,6))

In [ ]:
gdf.plot("area", legend=True, cmap='YlGn', figsize=(10,6))

In [ ]:
# o dinámicos (requiere la librería `folium`, comprobar todos los métodos instalados en la versión de geopandas con print(dir(gpd)) debe estar 'explore'.
# Si no está este método 'explore' intalar la última versión conda install --channel conda-forge geopandas==0.12.2)
gdf.explore("area", legend=False)

In [ ]:
# podemos dibujar las fronteras y los centroides como si fueran capas

map=gdf.plot("distancia", legend=True)
map=gdf.boundary.plot(ax=map, color="white", linewidth=.5)
map=gdf["centroide"].plot(ax=map, color="black")

Podemos cambiar la paleta de colores   [Paletas de Colores en matplotlib](https://matplotlib.org/stable/tutorials/colors/colormaps.html)


In [ ]:
gdf.plot(column='area', scheme='equal_interval', k=3, cmap='OrRd', edgecolor='k') 

O podemos añadir a un gráfico estático también fondos de OpenstreetMap y de otras librerías 

In [ ]:
# pip install contextily desde el terminal si fuese necesario
import contextily as cx
import xyzservices.providers as xyz
# pasamos nuestro mapa al sistema Web Mercator (EPSG 3857)
gdf_wm = gdf.to_crs(epsg=3857)
map_c=gdf_wm.plot( edgecolor='k', figsize=(10,7), alpha=0.25)
# Ahora le añadimos una cartografía base
cx.add_basemap(map_c, source=xyz.NASAGIBS.ViirsEarthAtNight2012, alpha=0.95)
               
# Buscar diferentes proveedores de cartografías:  https://contextily.readthedocs.io/en/latest/providers_deepdive.html

#cx.add_basemap(map_c, source=xyz.CartoDB.Positron)
#cx.add_basemap(map_c, source=xyz.OpenStreetMap.HOT)
#cx.add_basemap(map_c, source=xyz.OpenTopoMap)


## Importar Metadatos a una Cartografías

Ahora vamos a añadirle datos a cada CCAA


In [ ]:
# aquí estoy utilizando la librería Pandas as pd
salarios=pd.read_csv("datos/SALARIOS.csv",encoding='latin1',sep=";")
salarios.head()

In [ ]:
# ahora fusionamos los datos de salarios a la cartografía
gdf=pd.merge(gdf, salarios,how='left', left_on='cod_CCAA', right_on='COD_CCAA')
gdf



In [ ]:
ax=gdf.plot(column='SALARIO', 
         scheme='EqualInterval',
         k=8, cmap='YlOrRd',
         edgecolor='w',
         legend=True,
         figsize=(10,10),
         legend_kwds={'loc': 'lower right', 'fontsize':'8', 'title':'Salarios'} 
)
ax.set_axis_off()   

In [ ]:

ax=gdf.plot(column='SALARIO', 
         scheme='quantiles',
         k=8, cmap='YlOrRd',
         edgecolor='w',
         legend=True,
         figsize=(10,10),
         legend_kwds={'loc': 'lower right', 'fontsize':'8', 'title':'Salarios'} 
)
ax.set_axis_off()       


Para terminar voy a **grabar** la cartografía con todos los campos para no tener que volver a cargarla.   

**Ojo que por ejemplo el formato ESRI 'shp' sólo permite una cartografía**

In [ ]:
gdf.info()

Tengo que quitar una de las dos columans de cartografías

In [ ]:
gdf["xlong"]=gdf['centroide'].x
gdf["ylat"]=gdf['centroide'].y
gdf=gdf.drop('centroide', axis=1)
gdf.head()

In [ ]:
gdf.info()

In [ ]:
gdf.to_file('cartografias/CCAA_map.shp', encoding='latin1')

# Mapa de España

Ahora voy a hacer un mapa de España

In [ ]:

gdfm =gpd.read_file("cartografias/Munic04_ESP.shp")
gdfm.info()


In [ ]:
map=gdfm.plot(column='PrecioIn16', 
         scheme='quantiles',
         k=8, cmap='YlOrRd',
         legend=True,
         figsize=(10,10),
         legend_kwds={'loc': 'lower right', 'fontsize':'8', 'title':'Precio de la Vivienda'} 
)
map.set_axis_off()       


Puedo intentar ponerle encima cartografía de comunidades autónomas o provincias. Primero voy a crear la cartografía de las provincias agregando municipios (features) utilizando la función `dissolve()` de `GeoPandas`

In [ ]:
gprov=gdfm.dissolve(by="COD_PROV")
gprov.plot() 
   

In [ ]:
map=gdfm.plot(column='PrecioIn16', 
         scheme='quantiles',
         k=8, cmap='YlOrRd',
         legend=True,
         figsize=(10,10),
         legend_kwds={'loc': 'lower right', 'fontsize':'8', 'title':'Precio de la Vivienda'} 
)
map=gprov.boundary.plot(ax=map,color="dimgray", linewidth=.75)
map.set_axis_off()  

## Extensiones

Hay un gran número de paquetes y de utilidades para trabajar con datos espaciales. Por ejemplo:
- para poder representar puntos en un mapa es necesario tner la geolocalizaciópn de los puntos utilizando por ejemplo las direcciones postales. La función de `GeoPandas` **`tools.geocode()`** sirve para geolocalizar direcciones. También la librería **`geocoder`** hace lo mismo     
    
- La librería **`contextily`** sirve para añadir mapas contextuales OpenStreetMap a los gráficos de `GeoPandas`

- La librería **`osmnx`** sirve para interactuar con la api de Openstreetmap

- La librería **`Folium`** sirve para crear mapas web con Leaflet

- La librería **`Shapely`** para gestionar cartografías

